<a href="https://colab.research.google.com/github/picka-chu/Hype/blob/main/Aibet_Telegram_Football_Betting_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import telebot
from telebot import types
import json
import time
import logging
from datetime import datetime, timedelta

# --- Dependencies for Webhook & Deployment ---
from flask import Flask, request

# --- Dependencies for Firebase ---
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.exceptions import NotFound

# --- Configuration & Environment Variables ---
# NOTE: Replace these with your actual environment variable keys and values.
# On Pydroid 3, you would manually define these variables or use a .env solution.
try:
    # 1. Bot Token (from BotFather)
    BOT_TOKEN = os.environ.get('BOT_TOKEN')
    # 2. Telegram Payment Provider Token (e.g., from @BotFather, using a test token like Stripe)
    PROVIDER_TOKEN = os.environ.get('PROVIDER_TOKEN')
    # 3. Webhook URL Base (e.g., https://your-render-app-name.onrender.com)
    WEBHOOK_URL_BASE = os.environ.get('WEBHOOK_URL_BASE')
    # 4. A secret path component for the webhook URL
    WEBHOOK_PATH = f"/{os.environ.get('WEBHOOK_SECRET', 'aibet_secret_path')}"
    # 5. Firebase Credentials (JSON string or path to service account file)
    FIREBASE_CREDENTIALS = os.environ.get('FIREBASE_CREDENTIALS_JSON')

    if not all([BOT_TOKEN, PROVIDER_TOKEN, WEBHOOK_URL_BASE, FIREBASE_CREDENTIALS]):
        raise ValueError("One or more required environment variables are missing.")

except Exception as e:
    # Fallback for Pydroid 3 environment where os.environ might not be set easily
    print(f"Warning: Environment variables not found. Using placeholders. Error: {e}")
    BOT_TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"
    PROVIDER_TOKEN = "YOUR_TELEGRAM_PAYMENT_PROVIDER_TOKEN"  # Use a TEST token for development!
    WEBHOOK_URL_BASE = "YOUR_RENDER_APP_URL"
    WEBHOOK_PATH = "/aibet_secret_path"
    FIREBASE_CREDENTIALS = "{}" # Placeholder JSON string for the service account

# --- Constants & Global Setup ---
CURRENCY = "ETB"
BOT_USERNAME = "Aibet" # Replace with your bot's actual username

SUBSCRIPTION_PLANS = {
    "daily_plan": {"name": "1-Day Slip", "price": 50, "duration_days": 1},
    "weekly_plan": {"name": "1-Week Access", "price": 330, "duration_days": 7},
    "monthly_plan": {"name": "1-Month Access", "price": 1200, "duration_days": 30},
}
REFERRAL_REWARD_POINTS = 5
POINTS_PER_ETB = 10 # Example: 10 points = 1 ETB discount/value

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Firebase Initialization ---
try:
    if FIREBASE_CREDENTIALS:
        cred = credentials.Certificate(json.loads(FIREBASE_CREDENTIALS))
        firebase_admin.initialize_app(cred)
        db = firestore.client()
        logger.info("Firebase successfully initialized.")
    else:
        logger.error("Firebase credentials missing. DB functionality will be disabled.")
        db = None
except Exception as e:
    logger.error(f"Error initializing Firebase: {e}")
    db = None

# --- Telegram Bot Setup ---
bot = telebot.TeleBot(BOT_TOKEN, parse_mode='HTML', threaded=False)

# --- Database Management Class ---
class DBManager:
    """Handles all interaction with the Firestore Database."""

    def __init__(self, db_client):
        self.db = db_client
        self.users_collection = self.db.collection('aibet_users')
        self.slips_collection = self.db.collection('aibet_slips')

    def get_user(self, user_id):
        """Retrieves user data or returns None if not found/error."""
        try:
            doc_ref = self.users_collection.document(str(user_id))
            doc = doc_ref.get()
            if doc.exists:
                return doc.to_dict()
            return None
        except Exception as e:
            logger.error(f"DB Error (get_user {user_id}): {e}")
            return None

    def register_new_user(self, message, phone_number, referred_by_id=None):
        """Saves a new user to the database and grants free trial."""
        user_id = str(message.chat.id)
        current_time = time.time()

        # Grant 1-day free trial
        trial_expiry = current_time + (24 * 60 * 60) # 24 hours from now

        user_data = {
            'username': message.from_user.username or 'N/A',
            'full_name': (message.from_user.first_name or '') + (f" {message.from_user.last_name}" if message.from_user.last_name else ''),
            'telegram_id': user_id,
            'phone_number': phone_number,
            'subscription_status': 'free_trial',
            'trial_claimed': True,
            'expiry_date': trial_expiry,
            'referred_by_id': referred_by_id,
            'referral_points': 0,
            'slips_claimed_today': False,
            'registration_timestamp': current_time,
        }

        try:
            self.users_collection.document(user_id).set(user_data)
            logger.info(f"New user registered: {user_id}, Free Trial Granted until {datetime.fromtimestamp(trial_expiry)}")

            # Reward referrer if exists
            if referred_by_id and referred_by_id != user_id:
                self.add_referral_points(referred_by_id)

            return user_data
        except Exception as e:
            logger.error(f"DB Error (register_new_user {user_id}): {e}")
            return None

    def add_referral_points(self, referrer_id):
        """Increases referral points for the referrer."""
        try:
            doc_ref = self.users_collection.document(str(referrer_id))
            doc_ref.update({
                'referral_points': firestore.Increment(REFERRAL_REWARD_POINTS)
            })
            logger.info(f"Referrer {referrer_id} awarded {REFERRAL_REWARD_POINTS} points.")

            # Notify referrer (Optional, but good UX)
            bot.send_message(referrer_id, f"🌟 **Referral Reward!**\n\nSomeone you invited has joined! You've been awarded {REFERRAL_REWARD_POINTS} referral points.")

        except Exception as e:
            logger.error(f"DB Error (add_referral_points {referrer_id}): {e}")

    def update_subscription(self, user_id, plan_key):
        """Calculates new expiry date and updates subscription status after payment."""
        plan = SUBSCRIPTION_PLANS.get(plan_key)
        if not plan:
            logger.error(f"Invalid plan key provided: {plan_key}")
            return False

        user_data = self.get_user(user_id)
        if not user_data:
            logger.error(f"User not found for subscription update: {user_id}")
            return False

        duration_seconds = plan['duration_days'] * 24 * 60 * 60

        # Start expiry timer from now, or from the current expiry if subscription is still active.
        current_expiry = user_data.get('expiry_date', time.time())

        # Determine the start time for the new subscription
        # If current subscription is active, extend from the current expiry. Otherwise, start from now.
        start_time = max(time.time(), current_expiry)
        new_expiry = start_time + duration_seconds

        try:
            self.users_collection.document(str(user_id)).update({
                'subscription_status': 'active',
                'expiry_date': new_expiry,
                'trial_claimed': True, # Ensure trial is marked as used
            })
            logger.info(f"Subscription updated for {user_id}. New expiry: {datetime.fromtimestamp(new_expiry)}")
            return True
        except Exception as e:
            logger.error(f"DB Error (update_subscription {user_id}): {e}")
            return False

    def check_user_access(self, user_data):
        """Checks if a user is currently subscribed or on a free trial."""
        if not user_data:
            return False, "Not Registered"

        status = user_data.get('subscription_status', 'expired')
        expiry_date = user_data.get('expiry_date', 0)
        current_time = time.time()

        if status == 'active' or status == 'free_trial':
            if expiry_date > current_time:
                return True, status
            else:
                # Subscription expired, update status in DB
                self.users_collection.document(str(user_data['telegram_id'])).update({'subscription_status': 'expired'})
                return False, "Expired"

        return False, status

    def get_daily_slip(self):
        """Retrieves the latest bet slip (Admin functionality placeholder)."""
        # In a production app, the admin would upload the slip daily.
        # We look for a slip document keyed by today's date (YYYY-MM-DD).
        today_date = datetime.now().strftime('%Y-%m-%d')
        try:
            doc_ref = self.slips_collection.document(today_date)
            doc = doc_ref.get()
            if doc.exists:
                slip_data = doc.to_dict()
                return slip_data.get('slip_content', "No slip uploaded yet for today."), slip_data.get('odds', 'N/A')
            return None, None
        except Exception as e:
            logger.error(f"DB Error (get_daily_slip): {e}")
            return None, None

    # ADMIN ONLY: Placeholder function to upload a daily slip (for testing)
    def admin_upload_slip(self, content, odds):
        """(Admin) Uploads the daily slip."""
        today_date = datetime.now().strftime('%Y-%m-%d')
        try:
            self.slips_collection.document(today_date).set({
                'slip_content': content,
                'odds': odds,
                'upload_timestamp': time.time()
            })
            # Also reset everyone's 'slips_claimed_today' status to False for the new day
            # NOTE: For massive databases, this is inefficient. A nightly cron job is better.
            # For this context, we will skip mass update and assume the user can only claim one per day.
            logger.info(f"Admin uploaded slip for {today_date}. Content: {content[:30]}...")
            return True
        except Exception as e:
            logger.error(f"DB Error (admin_upload_slip): {e}")
            return False

    def mark_slip_claimed(self, user_id):
        """Marks the daily slip as claimed by the user."""
        try:
            self.users_collection.document(str(user_id)).update({'slips_claimed_today': True})
            logger.info(f"User {user_id} marked slip as claimed.")
            return True
        except Exception as e:
            logger.error(f"DB Error (mark_slip_claimed {user_id}): {e}")
            return False

# Initialize DB Manager if Firebase is active
db_manager = DBManager(db) if db else None

# --- Keyboard Functions ---
def get_main_menu_keyboard(is_registered):
    """Generates the main reply keyboard."""
    markup = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    if is_registered:
        # Registered User Menu
        markup.add(
            types.KeyboardButton("⚽ Daily Slip"),
            types.KeyboardButton("💳 My Account"),
            types.KeyboardButton("💰 Subscribe Now"),
            types.KeyboardButton("🤝 Refer & Earn"),
            types.KeyboardButton("/help")
        )
    else:
        # Registration Flow
        markup.add(
            types.KeyboardButton("📞 Share Contact", request_contact=True),
            types.KeyboardButton("/help")
        )
    return markup

def get_subscription_keyboard():
    """Generates inline keyboard for subscription plans."""
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton(f"Daily ({SUBSCRIPTION_PLANS['daily_plan']['price']} {CURRENCY})", callback_data='sub_daily_plan'))
    markup.add(types.InlineKeyboardButton(f"Weekly ({SUBSCRIPTION_PLANS['weekly_plan']['price']} {CURRENCY})", callback_data='sub_weekly_plan'))
    markup.add(types.InlineKeyboardButton(f"Monthly ({SUBSCRIPTION_PLANS['monthly_plan']['price']} {CURRENCY})", callback_data='sub_monthly_plan'))
    return markup

def get_referral_keyboard(user_id):
    """Generates an inline keyboard with the referral link."""
    referral_link = f"https://t.me/{BOT_USERNAME}?start={user_id}"
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Share Referral Link", url=f"tg://msg?text=Join%20Aibet%20for%20AI-driven%20football%20slips!%20Your%20link:%20{referral_link}"))
    markup.add(types.InlineKeyboardButton("Redeem Points", callback_data='redeem_points'))
    return markup

# --- CORE HANDLERS ---

@bot.message_handler(commands=['start'])
def handle_start(message):
    """Handles the /start command, including deep linking for referrals."""
    chat_id = message.chat.id
    user_id = str(chat_id)

    # Extract referral ID from deep link (e.g., /start ref_ID)
    referred_by_id = None
    if message.text and len(message.text.split()) > 1:
        # The second part of the message is the payload (ref_ID)
        payload = message.text.split()[1]
        if payload.isdigit():
            referred_by_id = payload if payload != user_id else None # Cannot refer self
            if referred_by_id:
                logger.info(f"User {user_id} started via referral link from {referred_by_id}")

    user_data = db_manager.get_user(user_id) if db_manager else None

    if user_data:
        # Existing user
        bot.send_message(
            chat_id,
            "👋 Welcome back to <b>Aibet!</b>\n\nUse the menu below to navigate your dashboard.",
            reply_markup=get_main_menu_keyboard(is_registered=True)
        )
        handle_my_account(message) # Show dashboard immediately
    else:
        # New user registration flow
        welcome_message = (
            "⚽ **Aibet: AI-Driven Verified Bet Slips (2.0+ Odds)**\n\n"
            "We provide daily, expertly reviewed football betting slips generated by advanced AI analysis.\n\n"
            "To get started and claim your **one-time, 24-hour Free Trial Slip**, please share your phone number.\n\n"
            "This number will be your account key. Tap the '📞 Share Contact' button below."
        )
        bot.send_message(
            chat_id,
            welcome_message,
            reply_markup=get_main_menu_keyboard(is_registered=False)
        )
        # Store referral ID in temporary state or context if needed, but for simplicity, we pass it during registration.
        if referred_by_id:
             bot.send_message(chat_id, f"🎉 You were invited by user ID: `{referred_by_id}`. You will both be rewarded when you register!")
             # Using bot's user_data to temporarily store the referrer ID for the next step (contact handler)
             bot.user_data[chat_id] = {'referred_by_id': referred_by_id}

@bot.message_handler(content_types=['contact'])
def handle_contact(message):
    """Handles phone number capture and finalizes user registration."""
    chat_id = message.chat.id
    user_id = str(chat_id)
    contact = message.contact

    if contact.user_id != chat_id:
        # Security check: ensure the contact belongs to the user
        bot.send_message(chat_id, "❌ Please share *your own* contact number using the button provided, not someone else's.")
        return

    phone_number = contact.phone_number

    # Check if user already exists based on Telegram ID (in case of double-send)
    user_data = db_manager.get_user(user_id) if db_manager else None
    if user_data:
        bot.send_message(chat_id, "You are already registered! Welcome back.", reply_markup=get_main_menu_keyboard(True))
        handle_my_account(message)
        return

    # Check for temporary referral ID
    referred_by_id = bot.user_data.pop(chat_id, {}).get('referred_by_id') if hasattr(bot, 'user_data') else None

    # Register the new user
    new_user_data = db_manager.register_new_user(message, phone_number, referred_by_id)

    if new_user_data:
        bot.send_message(
            chat_id,
            f"✅ **Registration Complete!**\n\n"
            f"You have successfully claimed your **24-hour Free Trial**.\n"
            f"Your access expires on: *{datetime.fromtimestamp(new_user_data['expiry_date']).strftime('%Y-%m-%d %H:%M:%S')}*.\n\n"
            "Use the '⚽ Daily Slip' button now!",
            reply_markup=get_main_menu_keyboard(is_registered=True)
        )
    else:
        bot.send_message(chat_id, "❌ **Registration Failed.**\n\nWe encountered a database error. Please try again later or contact support via /help.")

@bot.message_handler(func=lambda message: message.text in ["💳 My Account", "⚽ Daily Slip", "💰 Subscribe Now", "🤝 Refer & Earn", "/help"])
def handle_main_menu(message):
    """Handles the reply keyboard main menu interactions."""
    chat_id = message.chat.id
    user_id = str(chat_id)
    user_data = db_manager.get_user(user_id) if db_manager else None

    if not user_data:
        # Prompt for registration if not found
        handle_start(message)
        return

    text = message.text
    if text == "💳 My Account":
        handle_my_account(message)
    elif text == "⚽ Daily Slip":
        handle_daily_slip(message, user_data)
    elif text == "💰 Subscribe Now":
        handle_subscribe_menu(message)
    elif text == "🤝 Refer & Earn":
        handle_referral_menu(message, user_data)
    elif text == "/help":
        handle_help(message)

def handle_my_account(message):
    """Displays the user's detailed dashboard."""
    user_data = db_manager.get_user(str(message.chat.id))
    if not user_data:
        return # Handled by calling function

    has_access, status_code = db_manager.check_user_access(user_data)

    expiry_ts = user_data.get('expiry_date', 0)
    expiry_date_str = datetime.fromtimestamp(expiry_ts).strftime('%Y-%m-%d %H:%M:%S') if expiry_ts > time.time() else "N/A (EXPIRED)"

    access_emoji = "✅ ACTIVE" if has_access else "❌ EXPIRED"
    status_display = status_code.replace('_', ' ').title()

    dashboard_text = (
        f"👤 **Aibet Dashboard**\n\n"
        f"**Telegram ID:** `{user_data['telegram_id']}`\n"
        f"**Phone:** `{user_data['phone_number']}`\n\n"
        f"**Subscription Status:** {access_emoji}\n"
        f"   - Current Plan: *{status_display}*\n"
        f"   - Expiry Date: *{expiry_date_str}*\n\n"
        f"**Referral Points:** {user_data.get('referral_points', 0)} points\n\n"
        f"**Next Steps:**\n"
        f"   - Need a slip? Tap '⚽ Daily Slip'.\n"
        f"   - Extend access? Tap '💰 Subscribe Now'."
    )
    bot.send_message(message.chat.id, dashboard_text)

def handle_daily_slip(message, user_data):
    """Checks access and delivers the slip."""
    chat_id = message.chat.id
    user_id = str(chat_id)

    # 1. Check Subscription Status
    has_access, status_code = db_manager.check_user_access(user_data)

    if not has_access:
        bot.send_message(
            chat_id,
            "🔒 **Access Denied.**\n\nYour subscription has expired or you have not subscribed yet.\n"
            "Please tap '💰 Subscribe Now' to renew your access to the daily slips!",
            reply_markup=get_subscription_keyboard()
        )
        return

    # 2. Check if slip has already been claimed today
    if user_data.get('slips_claimed_today', False):
        bot.send_message(
            chat_id,
            "🕰️ **Slip Already Claimed.**\n\nYou have already received today's slip. Check back tomorrow for the new AI-verified selection!"
        )
        return

    # 3. Fetch the daily slip
    slip_content, odds = db_manager.get_daily_slip()

    if slip_content is None:
        bot.send_message(
            chat_id,
            "⚠️ **Slip Pending.**\n\nThe daily AI-verified slip (2.0+ odds) has not been uploaded by the experts yet. Please check back in a few hours! "
        )
        return

    # 4. Deliver the slip and mark as claimed
    slip_message = (
        f"🔥 **Aibet Daily Verified Slip** 🔥\n\n"
        f"**Odds:** `{odds}`+\n"
        f"**Date:** `{datetime.now().strftime('%Y-%m-%d')}`\n\n"
        f"--------------------------\n"
        f"{slip_content}\n"
        f"--------------------------\n\n"
        f"💸 Good luck! Your access is valid until: *{datetime.fromtimestamp(user_data['expiry_date']).strftime('%Y-%m-%d %H:%M:%S')}*"
    )
    bot.send_message(chat_id, slip_message)
    db_manager.mark_slip_claimed(user_id) # Mark as claimed

def handle_subscribe_menu(message):
    """Sends the subscription plan menu."""
    bot.send_message(
        message.chat.id,
        "✨ **Select Your Aibet Subscription Plan**\n\n"
        "All plans grant immediate, verified access to daily 2.0+ odds slips.\n\n"
        "1. Daily Slip: 50 ETB\n"
        "2. Weekly Access: 330 ETB\n"
        "3. Monthly Access: 1200 ETB",
        reply_markup=get_subscription_keyboard()
    )

def handle_referral_menu(message, user_data):
    """Displays referral info and options."""
    referral_points = user_data.get('referral_points', 0)
    points_value = referral_points / POINTS_PER_ETB

    referral_text = (
        f"🤝 **Aibet Referral & Rewards System**\n\n"
        f"**How it works:** Share your unique link. When someone registers, you earn **{REFERRAL_REWARD_POINTS}** points!\n"
        f"**10 Points = 1 {CURRENCY}** value.\n\n"
        f"**Your Current Points:** {referral_points} points ({points_value:.2f} {CURRENCY} value).\n\n"
        f"Tap 'Share Referral Link' below to get started, or 'Redeem Points' to use them for a subscription."
    )
    bot.send_message(message.chat.id, referral_text, reply_markup=get_referral_keyboard(user_data['telegram_id']))

@bot.message_handler(commands=['help'])
def handle_help(message):
    """Sends the help and guidance message."""
    help_text = (
        "❓ **Aibet Bot Help & Guide**\n\n"
        "**1. Core Functionality**\n"
        "   - **⚽ Daily Slip:** Delivers today's AI-verified bet slip (2.0+ odds). Requires an active subscription or free trial.\n"
        "   - **💳 My Account:** Shows your subscription status, expiry date, and referral points.\n\n"
        "**2. Subscriptions & Payment**\n"
        f"   - **💰 Subscribe Now:** View and select from our Daily (50 {CURRENCY}), Weekly (330 {CURRENCY}), or Monthly (1200 {CURRENCY}) plans.\n"
        "   - Payment is processed via secure Telegram Invoices (using a Telegram Payment Provider, e.g., Stripe/Ethiopian provider).\n"
        "   - You will be notified 3 days before expiry.\n\n"
        "**3. Referral System**\n"
        "   - **🤝 Refer & Earn:** Get your unique referral link. You earn points when a new user registers through your link.\n"
        f"   - Points can be redeemed for subscription plans (10 points = 1 {CURRENCY} value).\n\n"
        "**4. Troubleshooting**\n"
        "   - If payment fails, please retry. If the issue persists, ensure your payment provider is correctly set up with Telegram.\n"
        "   - For all other issues, please forward this message to an administrator."
    )
    bot.send_message(message.chat.id, help_text)

# --- CALLBACK QUERY HANDLER (For Subscription Selection) ---

@bot.callback_query_handler(func=lambda call: call.data.startswith('sub_'))
def handle_subscription_callback(call):
    """Handles inline button presses for selecting a subscription plan."""
    bot.answer_callback_query(call.id, "Preparing invoice...")
    chat_id = call.message.chat.id
    plan_key = call.data.replace('sub_', '')
    plan = SUBSCRIPTION_PLANS.get(plan_key)

    if not plan:
        bot.send_message(chat_id, "❌ Invalid plan selected. Please try again.")
        return

    # Check if a provider token is available
    if PROVIDER_TOKEN == "YOUR_TELEGRAM_PAYMENT_PROVIDER_TOKEN" or not PROVIDER_TOKEN:
        bot.send_message(chat_id, "❌ **Payment Not Available.**\n\nAdmin has not configured the Telegram Payment Provider Token yet. Please contact support or check back later.")
        return

    # Prepare the invoice
    price_in_cents = plan['price'] * 100

    invoice_payload = {
        "user_id": str(chat_id),
        "plan_key": plan_key,
        "timestamp": time.time()
    }

    try:
        bot.send_invoice(
            chat_id,
            title=plan['name'],
            description=f"Purchase {plan['name']} for AI-verified daily slips.",
            invoice_payload=json.dumps(invoice_payload),
            provider_token=PROVIDER_TOKEN,
            currency=CURRENCY,
            prices=[types.LabeledPrice(label=plan['name'], amount=price_in_cents)],
            start_parameter=plan_key,
            is_flexible=False # Simple, non-shipping payment
        )
    except Exception as e:
        logger.error(f"Failed to send invoice for {chat_id}: {e}")
        bot.send_message(chat_id, "❌ **Payment Error.**\n\nCould not generate the payment invoice. Please ensure the provider token is correct or contact support.")


# --- PAYMENT HANDLERS ---

@bot.pre_checkout_query_handler(func=lambda query: True)
def pre_checkout(pre_checkout_query):
    """Handles the pre-checkout confirmation from Telegram."""
    # Always answer True if the goods are available (which they are for digital subscriptions)
    try:
        # You could add logic here to check stock or user eligibility if necessary
        bot.answer_pre_checkout_query(pre_checkout_query.id, ok=True)
    except Exception as e:
        logger.error(f"Error answering pre_checkout_query for {pre_checkout_query.from_user.id}: {e}")

@bot.message_handler(content_types=['successful_payment'])
def successful_payment(message):
    """Handles the final confirmation after a successful payment."""
    chat_id = message.chat.id
    user_id = str(chat_id)

    try:
        # Extract the original payload
        invoice_payload = json.loads(message.successful_payment.invoice_payload)
        plan_key = invoice_payload.get('plan_key')

        if not plan_key or not db_manager.update_subscription(user_id, plan_key):
            bot.send_message(chat_id, "⚠️ **Subscription Update Failed.**\n\nPayment was successful, but the database could not be updated. Please contact support immediately with your Telegram ID.")
            return

        user_data = db_manager.get_user(user_id)
        if user_data:
            expiry_date_str = datetime.fromtimestamp(user_data['expiry_date']).strftime('%Y-%m-%d %H:%M:%S')

            success_message = (
                f"🎉 **Payment Successful!** 🎉\n\n"
                f"Thank you for purchasing the **{SUBSCRIPTION_PLANS[plan_key]['name']}**.\n"
                f"Amount Paid: `{message.successful_payment.total_amount / 100:.2f} {message.successful_payment.currency}`.\n\n"
                f"**Your Subscription is now ACTIVE!**\n"
                f"New Expiry Date: *{expiry_date_str}*\n\n"
                f"Tap '⚽ Daily Slip' to get today's AI-verified prediction."
            )
            bot.send_message(chat_id, success_message, reply_markup=get_main_menu_keyboard(True))

    except Exception as e:
        logger.error(f"Error processing successful payment for {user_id}: {e}")
        bot.send_message(chat_id, "❌ **Critical Error during Payment Confirmation.**\n\nSomething went wrong after a successful payment. Please contact support immediately.")

# --- Referral Redemption Handler (Placeholder for complex logic) ---

@bot.callback_query_handler(func=lambda call: call.data == 'redeem_points')
def handle_redeem_points(call):
    """Handles point redemption logic."""
    bot.answer_callback_query(call.id, "Initiating point redemption...")
    chat_id = call.message.chat.id
    user_id = str(chat_id)
    user_data = db_manager.get_user(user_id)

    if not user_data:
        bot.send_message(chat_id, "Please use /start to register first.")
        return

    points = user_data.get('referral_points', 0)

    # Example redemption: Redeem 1200 points for a 1-Month Plan (1200 ETB)
    MONTHLY_PLAN_POINTS = SUBSCRIPTION_PLANS['monthly_plan']['price'] * POINTS_PER_ETB # 1200 * 10 = 12000

    if points >= MONTHLY_PLAN_POINTS:
        # Deduct points and grant a monthly subscription
        try:
            db_manager.users_collection.document(user_id).update({
                'referral_points': firestore.Increment(-MONTHLY_PLAN_POINTS)
            })
            db_manager.update_subscription(user_id, 'monthly_plan')

            bot.send_message(
                chat_id,
                f"🥳 **Points Redeemed!**\n\n"
                f"You successfully redeemed {MONTHLY_PLAN_POINTS} points for a **1-Month Subscription!**\n"
                f"Your new expiry date is *{datetime.fromtimestamp(db_manager.get_user(user_id)['expiry_date']).strftime('%Y-%m-%d %H:%M:%S')}*."
            )
        except Exception as e:
            logger.error(f"Redemption error for {user_id}: {e}")
            bot.send_message(chat_id, "❌ Database error during redemption. Please try again.")

    else:
        points_needed = MONTHLY_PLAN_POINTS - points
        bot.send_message(
            chat_id,
            f"🚫 **Insufficient Points.**\n\n"
            f"You currently have {points} points. You need {MONTHLY_PLAN_POINTS} points for a 1-Month Plan redemption.\n"
            f"Keep referring users to earn {points_needed} more points!"
        )

# --- Admin Function (Simulated Daily Check/Reminder) ---
# NOTE: In a production environment on Render, this would run as a separate cron job.

@bot.message_handler(commands=['admin_check_expiry'])
def admin_check_expiry(message):
    """(Admin Only) Manually trigger the expiry check process."""
    if str(message.chat.id) != os.environ.get('ADMIN_TELEGRAM_ID'):
        bot.send_message(message.chat.id, "Access Denied.")
        return

    current_time = time.time()
    three_days = 3 * 24 * 60 * 60

    # Query users whose subscription expires in the next 3 days
    # NOTE: Firestore query limitations mean this needs external processing.
    # For a real-world bot, you would query and process the entire list of 'active' users.

    # SIMPLIFIED CHECK: Just check a few users or log a reminder for the admin
    logger.info("Admin manually triggered expiry check. In a real app, this runs as a scheduled job.")
    bot.send_message(message.chat.id, "Expiry check logging initiated. Check server logs for details.")

# --- Webhook and Flask Setup for Render Deployment ---
app = Flask(__name__)

@app.route(WEBHOOK_PATH, methods=['POST'])
def webhook():
    """Handles incoming Telegram updates via POST request."""
    if request.headers.get('content-type') == 'application/json':
        json_string = request.get_data().decode('utf-8')
        update = telebot.types.Update.de_json(json_string)

        try:
            bot.process_new_updates([update])
            return '', 200 # Success response
        except Exception as e:
            logger.error(f"Error processing update: {e}")
            return 'Update processing failed', 500
    else:
        # Invalid content type
        return 'Content-Type must be application/json', 403

@app.route('/')
def index():
    """Health check endpoint."""
    return f"Aibet Bot is running in Webhook mode at: {WEBHOOK_URL_BASE}{WEBHOOK_PATH}"

@app.route('/set_webhook')
def set_webhook_route():
    """Sets the Telegram webhook URL."""
    if BOT_TOKEN == "YOUR_TELEGRAM_BOT_TOKEN":
        return "ERROR: BOT_TOKEN is a placeholder. Cannot set webhook.", 200

    full_webhook_url = f"{WEBHOOK_URL_BASE}{WEBHOOK_PATH}"
    # Remove existing webhook and set the new one
    bot.remove_webhook()
    time.sleep(0.1) # Wait briefly
    success = bot.set_webhook(url=full_webhook_url)

    if success:
        logger.info(f"Webhook set successfully to: {full_webhook_url}")
        return f"Webhook set to {full_webhook_url}", 200
    else:
        logger.error(f"Failed to set webhook to: {full_webhook_url}")
        return "Failed to set webhook.", 500

# --- Main Application Execution (for Pydroid 3 or local testing) ---
if __name__ == '__main__':
    # You would typically run the bot in webhook mode on a server like Render.
    # For local Pydroid 3 development, you might switch to polling mode:
    # bot.infinity_polling()

    # For deployment on Render, run the Flask app:
    port = int(os.environ.get('PORT', 5000))
    # It's recommended to call /set_webhook immediately after deployment
    # Example: access the /set_webhook endpoint manually after deploying

    # For local testing, you must mock Firebase and run in polling mode:
    # if BOT_TOKEN != "YOUR_TELEGRAM_BOT_TOKEN" and PROVIDER_TOKEN != "YOUR_TELEGRAM_PAYMENT_PROVIDER_TOKEN":
    #    bot.remove_webhook()
    #    bot.infinity_polling()
    # else:
    #    print("Bot is running in Flask mode on port 5000. Use /set_webhook URL manually to connect Telegram.")

    app.run(host='0.0.0.0', port=port)

# --- Admin Placeholder (Run this manually in Pydroid/local env for testing) ---
# db_manager.admin_upload_slip("Bet: Man Utd vs Liverpool - Over 2.5 Goals\nReason: High scoring history, both teams in form.", 2.15)
# print("Test slip uploaded for today.")